Connect Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

From https://www.youtube.com/watch?v=YjWh7QvVH60

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

CNN Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms

device = torch.device('cpu')
# Output height = (Input height + padding height top + padding height bottom - kernel height) / (stride height) + 1
# Output width  = (Input width + padding width right + padding width left - kernel width) / (stride width) + 1

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # image shape is 3 * 40 * 40, where 3 is one color channel
        # 40 * 40 is the image size
        # 256* 256
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)    # # output shape = 6 * 252 * 252
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)                       # output shape = 6 * 126 * 126
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=9, kernel_size=5)    # output shape = 9 * 122 * 122
        #nn.MaxPool2d(kernel_size=2, stride=2)                                  # output shape = 9 * 61 * 61
        
        # output shape=  9 * 7 * 7
        self.fc1 = nn.Linear(9*61*61, 100)
        self.fc2 = nn.Linear(100, 50)
        
        # last fully connected layer output should be same as classes
        self.fc3 = nn.Linear(50, 2)

        
    def forward(self, x):
        # first conv
        x = self.pool(F.relu(self.conv1(x)))
        # second conv
        x = self.pool(F.relu(self.conv2(x)))
        # flatten all dimensions except batch
        x = torch.flatten(x, 1)
        # fully connected layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [ ]:
model = ConvNet()
model.to(device)

model.load_state_dict(torch.load("drive/MyDrive/CNN/basic_CNN.model",map_location='cpu'))
model.eval()

Model classifies.

In [ ]:
def m_or_f(img):
  img = cv2.resize(img, (256, 256))
  img = img.reshape(3, 256, 256).astype("float32")
  img = img / 255
  tran = transforms.ToTensor()
  img = tran(img)
  
  img = img.to(device)
  img = img.view(1, 3, 256, 256)
  out1 = model(img)
  out1 = F.softmax(out1, dim = 1)
  proba, class_ind = torch.max(out1, 1)
  class_ind = int(class_ind)
  proba = float(proba)
  return class_ind, out1

WebCam + classification

In [19]:
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
with torch.no_grad():
  while True:
      js_reply = video_frame(label_html, bbox)
      if not js_reply:
          break
      
      stream = js_to_image(js_reply["img"])

      face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + "haarcascade_frontalface_default.xml"))
      eye_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + "haarcascade_eye.xml"))

      #faces = face_cascade.detectMultiScale(stream,scaleFactor=1.2, minNeighbors=2, minSize=(32,32))
      img = stream
      bbox_array = np.zeros([480,640,4], dtype=np.uint8)
      eye_bbox_array = np.zeros([480,640,4], dtype=np.uint8)

      eyes = eye_cascade.detectMultiScale(stream, 1.3, 10, minSize=(32,32))
      for (ex,ey,ew,eh) in eyes:
        #print(img[ey:ey+eh, ex:ex+ew].shape)
        output1, output2 = m_or_f(img[ey:ey+eh, ex:ex+ew])
        print(output2)
        if output1 == 0:
          eye_bbox_array = cv2.rectangle(eye_bbox_array, (ex,ey), (ex+ew,ey+eh), (0,0,255), 1)
        else:
          eye_bbox_array = cv2.rectangle(eye_bbox_array, (ex,ey), (ex+ew,ey+eh), (255,0,0), 1)
      eye_bbox_array[:,:,3] = (eye_bbox_array.max(axis = 2) > 0 ).astype(int) * 255
      eye_bbox_bytes = bbox_to_bytes(eye_bbox_array) 
      bbox = eye_bbox_bytes


<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored